## Vgg16BN Finetuning experienced with the batch_normalized fc layers

### Result: Can't get val_loss to come down below what we had for regular Vgg16
###              Currently unable to rewire the trained dense-layers back with the conv.
###              model to generate outputs.

In [2]:
%load_ext autoreload

In [3]:
%autoreload Executor

In [4]:
%aimport Executor
%aimport Vgg16
%aimport Vgg16BN

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [6]:
from Executor import *

In [51]:
executor = ExecutorBuilder().\
    with_runID("trial.bn").\
    and_().\
    with_Vgg16BN().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()


Pretrained Vgg16BN model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [52]:
#done..
#executor.precompute_conv_model_outputs()
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7fb3d8bfcb00>

In [20]:
#executor.tune_softmax_layer_for_epochs(1)
# 6255/6255 [==============================] - 201s - loss: 2.3536 - acc: 0.4571 - val_loss: 1.1148 - val_acc: 0.5261


In [53]:
executor.conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]  

In [29]:
from keras.layers import Dense, BatchNormalization, Dropout, Lambda, Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential

def get_new_fc_model(conv_layer, num_softmax_classes, new_dropout=0.5):
    
    new_model = Sequential([
        MaxPooling2D(input_shape=conv_layer.output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(new_dropout),        
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(new_dropout),
        Dense(num_softmax_classes, activation='softmax')
    ])
        
    return new_model
fc_model = get_new_fc_model(executor.conv_layers[-1], 3)

In [30]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_11 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_5[0][0]       
____________________________________________________________________________________________________
flatten_7 (Flatten)              (None, 25088)         0           maxpooling2d_11[0][0]            
____________________________________________________________________________________________________
dense_18 (Dense)                 (None, 4096)          102764544   flatten_7[0][0]                  
____________________________________________________________________________________________________
batchnormalization_11 (BatchNorma(None, 4096)          8192        dense_18[0][0]                   
___________________________________________________________________________________________

In [31]:
new_dropout = 0.5
def proc_wgts(layer, prev_dropout, new_dropout):
    '''
    evaluates an array of rescaled weights

    :param layer: fc layer
    :param prev_dropout: prev dropout with which the layer was trained
    :param new_dropout:  new dropout to apply
    :return: array of rescaled weights
    '''
    scale = (1.0 - prev_dropout)/(1.0 - new_dropout)
    return [o * scale for o in layer.get_weights()]

for l1, l2 in zip(fc_model.layers, executor.fc_layers):
    print("found dense layer. Distributing scaled weights..")
    l1.set_weights(proc_wgts(l2, executor.dropout, new_dropout))


found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..


In [32]:
for layer in fc_model.layers:
    layer.trainable = False

fc_model.layers[-1].trainable = True

In [33]:
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 2.6158 - acc: 0.4523 - val_loss: 1.6641 - val_acc: 0.5061
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 2.0862 - acc: 0.4831 - val_loss: 1.1199 - val_acc: 0.5184
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.8127 - acc: 0.5060 - val_loss: 1.0485 - val_acc: 0.5462
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.6101 - acc: 0.5090 - val_loss: 1.0622 - val_acc: 0.5352
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.5365 - acc: 0.5173 - val_loss: 1.0533 - val_acc: 0.5313


In [34]:
# tune for 5 more times...
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.4151 - acc: 0.5303 - val_loss: 1.0025 - val_acc: 0.5455
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.3129 - acc: 0.5388 - val_loss: 0.9679 - val_acc: 0.5623
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.2642 - acc: 0.5412 - val_loss: 0.9863 - val_acc: 0.5623
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.2286 - acc: 0.5464 - val_loss: 0.9758 - val_acc: 0.5668
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.2689 - acc: 0.5442 - val_loss: 0.9547 - val_acc: 0.5662


In [35]:
# Tune for 5 more times with reduced learn_rate
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.1093 - acc: 0.5706 - val_loss: 0.9020 - val_acc: 0.5868
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.0827 - acc: 0.5664 - val_loss: 0.8917 - val_acc: 0.5901
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.0617 - acc: 0.5693 - val_loss: 0.8866 - val_acc: 0.5855
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.0495 - acc: 0.5680 - val_loss: 0.8752 - val_acc: 0.5907
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.0231 - acc: 0.5746 - val_loss: 0.8782 - val_acc: 0.5946


In [36]:
# Now set the entire dense layers as being trainable, and run the experiments:
for layer in fc_model.layers:
    layer.trainable=True
    
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 0.9979 - acc: 0.5861 - val_loss: 0.8362 - val_acc: 0.6185
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.8021 - acc: 0.6686 - val_loss: 0.8045 - val_acc: 0.6469
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.6681 - acc: 0.7159 - val_loss: 0.7719 - val_acc: 0.6637
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.5713 - acc: 0.7648 - val_loss: 0.7622 - val_acc: 0.6682
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.4696 - acc: 0.8094 - val_loss: 0.7392 - val_acc: 0.6862


In [37]:
# 5 more epochs at same lr
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))


fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 0.3843 - acc: 0.8481 - val_loss: 0.7373 - val_acc: 0.6908
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.3296 - acc: 0.8745 - val_loss: 0.7415 - val_acc: 0.6875
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.2614 - acc: 0.9062 - val_loss: 0.7367 - val_acc: 0.6946
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.2385 - acc: 0.9172 - val_loss: 0.7396 - val_acc: 0.6966
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.1866 - acc: 0.9359 - val_loss: 0.7459 - val_acc: 0.7011


In [38]:
# final val_loss increases.. so decrease lr
opt = Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 0.1582 - acc: 0.9493 - val_loss: 0.7470 - val_acc: 0.6998
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.1514 - acc: 0.9492 - val_loss: 0.7481 - val_acc: 0.7011
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.1488 - acc: 0.9504 - val_loss: 0.7484 - val_acc: 0.7017
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.1396 - acc: 0.9567 - val_loss: 0.7496 - val_acc: 0.7037
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.1387 - acc: 0.9562 - val_loss: 0.7526 - val_acc: 0.7017


In [54]:
executor.conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]  

In [56]:
executor.vgg.model =  executor.conv_model

In [57]:
executor.vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]  

In [58]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_11 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_5[0][0]       
____________________________________________________________________________________________________
flatten_7 (Flatten)              (None, 25088)         0           maxpooling2d_11[0][0]            
____________________________________________________________________________________________________
dense_18 (Dense)                 (None, 4096)          102764544   flatten_7[0][0]                  
____________________________________________________________________________________________________
batchnormalization_11 (BatchNorma(None, 4096)          8192        dense_18[0][0]                   
___________________________________________________________________________________________

## Can't seem to get past this !! Will get back to this later!

In [59]:
for layer in fc_model.layers:
    print("adding layer: ",layer)
    executor.vgg.model.add(layer)
    print("added..")

('adding layer: ', <keras.layers.pooling.MaxPooling2D object at 0x7fb3d6e06750>)
added..
('adding layer: ', <keras.layers.core.Flatten object at 0x7fb3d6e067d0>)
added..
('adding layer: ', <keras.layers.core.Dense object at 0x7fb3d6e06690>)
added..
('adding layer: ', <keras.layers.normalization.BatchNormalization object at 0x7fb3d6e06850>)


Exception: You are attempting to share a same `BatchNormalization` layer across different data flows. This is not possible. You should use `mode=2` in `BatchNormalization`, which has a similar behavior but is shareable (see docs for a description of the behavior).